In [30]:
#Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps

#Import API key
from API_Keys import gkey

#Configure gmaps
gmaps.configure(api_key=gkey)

In [31]:
#read in CSV file created in first part of homework
cities_df = pd.read_csv("Resources/cities.csv")
cities_df.head()

,city,country,latitude,longitude,temperature,humidity,cloudiness,wind speed
0,North Platte,US,41.12,-100.77,65.08,82,90,5.88
1,Busselton,AU,-33.65,115.33,41.50,67,0,7.20
2,Bredasdorp,ZA,-34.53,20.04,64.40,77,1,8.34
3,Jamestown,US,42.10,-79.24,72.61,77,1,1.34
4,Longyearbyen,SJ,78.22,15.64,42.80,75,40,21.92


In [32]:
#Create heat map of humidity for cities in dataframe
locations = cities_df[["latitude", "longitude"]]
humidity = cities_df["humidity"]

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [33]:
#Narrow data frame to my ideal weather conditions:
#  temperature between 70 and 80 degrees
#  humidity between 30 and 50 percent
#  cloudiness less than 30%

my_cities_df = cities_df.loc[(cities_df["temperature"] >= 70) & (cities_df["temperature"] <= 80) & 
                             (cities_df["humidity"] >= 30) & (cities_df["humidity"] <= 50) &
                             (cities_df["cloudiness"] < 30), :].reset_index()

#check number of cities in my ideal consideration set to see if a reasonable number
len(my_cities_df.index)

7

In [34]:
#Use Google Places API to find first hotel for each city located within 5000 meters of city coordinates

#set up additional column to hold hotel information
my_cities_df['hotel'] = ""

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"radius": 5000,
         "type": "lodging",
         "key": gkey,
         }

for index, row in my_cities_df.iterrows():
    
    #get location from dataframe
    lat = row['latitude']
    lon = row['longitude']
    params['location'] = f"{lat},{lon}"
    print(params["location"])
    
    #assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    #extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        
        my_cities_df.loc[index, 'hotel'] = results[0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("-----------------")
        

28.33,-105.4
Closest hotel is Hacienda Las Marias.
-----------------
70.63,147.92
Missing field/result... skipping.
-----------------
35.23,-106.66
Closest hotel is Hilton Garden Inn Albuquerque North/Rio Rancho.
-----------------
-22.12,14.28
Closest hotel is Fishermans Guest House.
-----------------
40.5,20.23
Closest hotel is Hotel Shpetimi.
-----------------
40.16,-8.33
Closest hotel is Casa Maquia.
-----------------
46.42,3.91
Closest hotel is Manoir Du Bonheur.
-----------------


In [35]:
#check data frame to see if hotels added
my_cities_df

,index,city,country,latitude,longitude,temperature,humidity,cloudiness,wind speed,hotel
0,13,Jiménez,MX,28.33,-105.40,77.00,31,20,5.82,Hacienda Las Marias
1,69,Chokurdakh,RU,70.63,147.92,75.65,34,1,7.52,
2,154,Rio Rancho,US,35.23,-106.66,71.78,35,0,0.89,Hilton Garden Inn Albuquerque North/Rio Rancho
3,168,Henties Bay,NaN,-22.12,14.28,75.99,37,29,1.99,Fishermans Guest House
4,371,Çorovodë,AL,40.50,20.23,76.01,49,2,10.04,Hotel Shpetimi
5,383,Lata,PT,40.16,-8.33,79.83,46,0,11.88,Casa Maquia
6,417,Saint-François,FR,46.42,3.91,74.14,40,28,5.82,Manoir Du Bonheur


In [36]:
#create a dictionary from data frame to use for marker info box
mydict = my_cities_df.to_dict('records')

In [37]:
#Plot hotels on top of humidity heat map with each pin containing the Hotel Name, City, and Country

#marker locations and info box
marker_locations = my_cities_df[["latitude", "longitude"]]

info_box_template = """
<dl>
<dt>Hotel</dt><dd>{hotel}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""

hotel_info = [info_box_template.format(**hotels) for hotels in mydict]


#create map
fig = gmaps.figure()

#heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=3)
fig.add_layer(heat_layer)

#marker layer
marker_layer = gmaps.marker_layer(marker_locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

fig


Figure(layout=FigureLayout(height='420px'))